# File

In [1]:
%%writefile automaticSeeding.py
#from read_files import normalization
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.io as spio

def find_seed(volume, thresh, lung):

    interval = volume.copy()
    cols=0
    if lung == 'left':
        cols = volume.shape[2]*2//3
    elif lung == 'right':
        cols = volume.shape[2]//3

    inicial_slices= volume.shape[0]*2//3
    slices=inicial_slices
    rows= volume.shape[1]//2
    inicial_seed = tuple([slices,rows,cols])
    mask_HU = np.zeros(volume.shape)
    min_HU = -800 #-850
    max_HU = -500#-700

    sup = abs(min_HU-thresh)
    inf = abs(max_HU+thresh)
    mask_HU =np.where((volume >= min_HU) & (volume <= max_HU),volume,False)
    #print('min: -'+str(inf))
    #print('max: -'+str(sup))
    ready = False
    second_round= False
    i = 0
    while ready == False:
        value = abs(volume[slices,rows,cols] )
        if value >= inf and value <= sup:
    
            interval[slices,rows,cols]=5000
            seed = tuple([slices,rows,cols])
            ready = True
            #print("I found one")
        else: 
            interval[slices,rows,cols]=5000
            slices=slices+1
            
            if slices>=volume.shape[0]: 
                #print("no seed was found")
                ready = True
                seed = [0,0,0]
    
#     fig = plt.figure()  
#     a = fig.add_subplot(1,1,1)
#     a.imshow(interval[:,rows,:]) 
#     plt.show()
    
    if seed == [0,0,0]:
        interval = volume.copy()
        slices = inicial_slices
        while second_round == False:
            #print('superiot to inf'+str(volume[inicial_slices,rows,cols] >= inf))
            #print('inferior to sup'+str(volume[inicial_slices,rows,cols] <= sup))
            value = abs(volume[slices,rows,cols] )
            #print('value'+str(value))
            
            if value >= inf and value <= sup:
                interval[inicial_slices,rows,cols]=5000
                seed = tuple([slices,rows,cols])
                second_round = True
            else: 
                interval[slices,rows,cols]=5000
                slices=slices-1
                #cols=cols
                if slices<=0:
                    print("no seed was found")
                    second_round = True
                    seed = [0,0,0]
    
#     fi = plt.figure()  
#     b = fi.add_subplot(1,1,1)
#     b.imshow(interval[:,rows,:]) 
#     plt.show()
    
    new_seed = tuple(seed)
    
#     fig_1 = plt.figure()  
#     fig_1.suptitle("Inicial seed", fontsize=16)
#     y = fig_1.add_subplot(1,2,1)
#     y.imshow(volume[:,rows,:])
#     y.scatter([inicial_seed[2]], [inicial_seed[0]],c='r', s=10)
#     x = fig_1.add_subplot(1,2,2)
#     x.imshow(mask_HU[:,rows,:])
#     x.scatter([inicial_seed[2]], [inicial_seed[0]],c='r',s=10)
#     plt.show()
    
#     fig_2 = plt.figure()  
#     fig_2.suptitle("Final Seed", fontsize=16)
#     y = fig_2.add_subplot(1,2,1)
#     y.imshow(volume[:,rows,:])
#     y.scatter([new_seed[2]], [new_seed[0]],c='r', s=10)
#     x = fig_2.add_subplot(1,2,2)
#     x.imshow(mask_HU[:,rows,:])
#     x.scatter([new_seed[2]], [new_seed[0]],c='r',s=10)
#     plt.show()

    return seed


Overwriting automaticSeeding.py


# Tests

In [2]:
%matplotlib inline

In [ ]:
resized_array = np.load("resized_array.npy")
seed = find_seed(resized_array,10,"right")
print(seed)

In [ ]:
import imutils
slices= resized_array.shape[0]//3
rows= resized_array.shape[1]//2
cols = resized_array.shape[2]//3

resized_array[slices]=5000
resized_array[:,:,cols]=5000
plt.imshow(imutils.rotate(resized_array[:,rows,:],180))

In [6]:
from segmentation_functions import resample, grow
from automaticSeeding import find_seed
from .finding_biggest_lung import arrange_slices, normalization, get_pixels_hu
#from read_files import normalization
import scipy.ndimage.interpolation as inter
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.io as spio
import pydicom
import time

slices_path = "G:/CTimages/"
indices_path ="G:/Plans/CT/Lungs/ok/Separated Lungs"
segmentations_results_path = "G:/Plans/CT/Lungs/segmentation_results/"
indices_folder= os.listdir(indices_path)

patient_id = "10194022"#each_mask.split(" ", each_mask.count(each_mask))[0] 
slices= arrange_slices(slices_path,patient_id)
normalized_volume=normalization(slices)
normalized_array = np.array(normalized_volume)
volume_resampled, spacing = resample(normalized_array, slices, [5,5,5])
seed = find_seed(volume_resampled, 50,"left")